In [ ]:
# Running out of http://192.168.1.44:8888/notebooks/YouTube/Version%202/

In [16]:
#pip install youtube_transcript_api

In [14]:
#from youtubesearchpython.__future__ import VideosSearch
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd
from os import listdir
from os import remove
import csv
import time

pd.set_option("display.max_rows", None)

In [15]:
def getTranscripts(video, file_name, id_file_name):
    subs = []
    retry = 0
    phrase = ""

    while len(subs) == 0 and retry < 10:
        try:
            subs=YouTubeTranscriptApi.get_transcript(video)
            retry += 1
        except:
            retry += 1

    for text in subs:
        oneLine = text["text"].replace("\n", " ")
        oneLine = " ".join(oneLine.split())

        if len(phrase) > 0:
            phrase = phrase + " " + oneLine.strip()
        else:
            phrase = oneLine.strip()

    if len(phrase) > 0:
        with open(file_name, mode="a") as out_file:
            result_writer = csv.writer(out_file, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
            result_writer.writerow([video, phrase])
            
        with open(id_file_name, mode="a") as id_out_file:
            id_result_writer = csv.writer(id_out_file, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
            id_result_writer.writerow([video])

In [16]:
# Create output files names
date = time.strftime("%Y-%m-%d_%H:%M:%S")
file_name = "processed/unique_videos_transcripts" + date + ".csv"
id_file_name = "processed/unique_videos_ids" + date + ".csv"

# Get videos
filepaths = ["processed/" + f for f in listdir("processed/") if f.startswith("unique_videos_metadata")]
df_videos = pd.concat(map(pd.read_csv, filepaths))

# Create a copy of the dataframe with unique videos
df_new_unique_videos = df_videos[["video"]].copy()

# Get already processed videos
filepaths = ["processed/" + f for f in listdir("processed/") if f.startswith("unique_videos_ids")]
if filepaths:
    df_processed = pd.concat(map(pd.read_csv, filepaths))
    df_processed.to_csv(id_file_name, index=False)
    for f in filepaths:
        remove(f)
    
    # Remove previously processed videos
    if len(df_processed) > 0:
        #df_processed = df_processed[["video"]]
        #cond = df_new_unique_videos["video"].isin(df_processed["video"])
        #df_new_unique_videos.drop(df_new_unique_videos[cond].index, inplace = True)
        df_new_unique_videos = pd.merge(df_new_unique_videos,df_processed, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)

else:
    with open(file_name, mode='a') as out_file:
        result_writer = csv.writer(out_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        result_writer.writerow(["video", "transcript"])

    with open(id_file_name, mode='a') as id_out_file:
        id_result_writer = csv.writer(id_out_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        id_result_writer.writerow(["video"])

#df_new_unique_videos = df_new_unique_videos[:50]

df_new_unique_videos = pd.DataFrame(df_new_unique_videos)
df_new_unique_videos = df_new_unique_videos.reindex(index=df_new_unique_videos.index[::-1])

print("New videos:", len(df_new_unique_videos))

i = 0
for index, row in df_new_unique_videos.iterrows():
    getTranscripts(row["video"], file_name, id_file_name)
    if i % 500 == 0: 
        print(i)
    i += 1

New videos: 39235
0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
